In [ ]:
import scanpy as sc
import squidpy as sq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import Counter
from matplotlib_scalebar.scalebar import ScaleBar
import os

import cell2location
import scvi

DPI = 300
FONTSIZE = 20  # 42
sc.settings.set_figure_params(
    scanpy=True, dpi=100, transparent=True, vector_friendly=True, dpi_save=DPI
)
from matplotlib import rcParams

rcParams["pdf.fonttype"] = 42

import seaborn as sns
from pathlib import Path

custom_params = {"font.size": FONTSIZE, "figure.dpi": 100, "savefig.dpi": DPI}
sns.set_theme(style="ticks", rc=custom_params)

# QC utils functions - a package
from vistools import utils
from vistools import spatial_plots
from vistools import cell2loc_spatial_colours

In [ ]:
SAMPLE_NAME = "concat_withWu2022"

# DIR2SAVE
DIR2SAVE = Path(
    f"/data/BCI-CRC/nasrine/data/CRC/spatial/CRC_LM_VISIUM/CRC_LM_VISIUM_04_08_09_11/cell2location/{SAMPLE_NAME}/"
)
DIR2SAVE.mkdir(parents=True, exist_ok=True)

run_name = DIR2SAVE.joinpath(f"cell2location_map-no_cycling_TME/")
run_name.mkdir(parents=True, exist_ok=True)

# figures
FIG2SAVE = run_name.joinpath("figures/final/")
FIG2SAVE.mkdir(parents=True, exist_ok=True)
# set the global variable: sc.settings.figdir to save all plots
sc.settings.figdir = FIG2SAVE

In [ ]:
adata_file = f"{run_name}/sp.h5ad"
adata_vis = sc.read_h5ad(adata_file)
mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

### Visualising cell abundance in spatial coordinates

We use 5% quantile of the posterior distribution, representing the value of cell abundance that the model has high confidence in (aka ‘at least this amount is present’).

In [ ]:
# add 5% quantile, representing confident cell abundance, 'at least this amount is present',
# to adata.obs with nice names for plotting
adata_vis.obs[adata_vis.uns["mod"]["factor_names"]] = adata_vis.obsm[
    "q05_cell_abundance_w_sf"
]
adata_vis.uns["mod"]["factor_names"]

In [ ]:
cell2plot = "Enteroendocrine"

In [ ]:
for visium_slide in set(adata_vis.obs["Sample"]):
    print(visium_slide)

    # select one slide
    slide = utils.select_slide(adata_vis, s=visium_slide, batch_key="Sample")
    
    # get size pixel
    lib_id = [x for x in slide.uns['spatial'].keys()][0]
    pix_size = spatial_plots.get_pixel_size_visium(slide, lib_id)

    # visualise cell locations
    # plot in spatial coordinates
    fig, ax = plt.subplots()
    with mpl.rc_context({"axes.facecolor": "black", "figure.figsize": [4.5, 5]}):

        sc.pl.spatial(
            slide,
            cmap="Blues", #plasma RdPu PuBu
            color=cell2plot,
            ncols=4,
            size=1.3,
            img_key="hires",
            alpha_img=0.5,
            # limit color scale at 99.2% quantile of cell abundance
            vmin=0,
            vmax="p99.4",
            #save=f"{visium_slide}_{cell2plot}_4HE.pdf",
            show=False,
            ax=ax
        )
    # add scale bar
    #scalebar = ScaleBar(pix_size, "um", length_fraction=0.25, frameon=False, location='lower right')
    #ax.add_artist(scalebar)
    
    fig.savefig(
            FIG2SAVE.joinpath(f"{visium_slide}_{cell2plot}.pdf"),
            dpi=DPI,
            format="pdf",
            facecolor="w",
            transparent=True,
            edgecolor="w",
            bbox_inches="tight"
        )  

In [ ]:
for visium_slide in set(adata_vis.obs["Sample"]):
    print(visium_slide)

    # select one slide
    slide = utils.select_slide(adata_vis, s=visium_slide, batch_key="Sample")
        
    sc.pl.spatial(slide,cmap='Blues',color=["Stem", "Stem (NOTUM high)", "pEMT", "ipEMT", "Hypoxia"
                                           ], 
                  wspace=0.2,ncols=5,
                  size=1.3, img_key='hires',
                  alpha_img=0.3, vmin=0, vmax='p99.4',
                  title=["Stem", "Stem NOTUM", "pEMT", "ipEMT", "Hypoxia"],
                  save=f"{visium_slide}_stem_pEMT_ipEMT.pdf"
                     )

### get HE and scale bar

In [ ]:
# assign alpha image for samples 
set(adata_vis.obs["Sample"])
alpha_name_dict = {
    'CRC08_LM_VISIUM': (1, "CRC08"),
    'CRC09_LM_VISIUM': (1, "CRC09"),
    'CRC11_LM_VISIUM': (1, "CRC11"),
    'P13_Liver_ST': (0.7, "P13"),
    'P3_liver': (0.7, "P3"),
    'ST_LM_4': (0.7, "LM4"),
    
}

In [ ]:
for visium_slide in set(adata_vis.obs["Sample"]):
    print(visium_slide)

    # select one slide
    slide = utils.select_slide(adata_vis, s=visium_slide, batch_key="Sample")
    
    # get size pixel
    lib_id = [x for x in slide.uns['spatial'].keys()][0]
    pix_size = spatial_plots.get_pixel_size_visium(slide, lib_id)

    # visualise cell locations
    # plot in spatial coordinates
    fig, ax = plt.subplots()
    with mpl.rc_context({"axes.facecolor": "black", "figure.figsize": [4.5, 5]}):

        sc.pl.spatial(
            slide,
            cmap="plasma",
            color=cell2plot,
            ncols=4,
            size=0,
            img_key="hires",
            alpha_img=alpha_name_dict[visium_slide][0],
            # limit color scale at 99.2% quantile of cell abundance
            vmin=0,
            vmax="p99.4",
            #save=f"{visium_slide}_{cell2plot}_4HE.pdf",
            title=f"{alpha_name_dict[visium_slide][1]}, H&E",
            show=False,
            ax=ax
        )
    # add scale bar
    scalebar = ScaleBar(pix_size, "um", length_fraction=0.25, frameon=False, location='lower right')
    ax.add_artist(scalebar)
    
    fig.savefig(
            FIG2SAVE.joinpath(f"{visium_slide}_HE.pdf"),
            dpi=DPI,
            format="pdf",
            facecolor="w",
            transparent=True,
            edgecolor="w",
            bbox_inches="tight"
        )  

In [ ]:
FIG2SAVE